## EDA And Feature Engineering Flight Price Prediction
check the dataset info below
https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction

### FEATURES
The various features of the cleaned dataset are explained below:
1) Airline: The name of the airline company is stored in the airline column. It is a categorical feature having 6 different airlines.
2) Flight: Flight stores information regarding the plane's flight code. It is a categorical feature.
3) Source City: City from which the flight takes off. It is a categorical feature having 6 unique cities.
4) Departure Time: This is a derived categorical feature obtained created by grouping time periods into bins. It stores information about the departure time and have 6 unique time labels.
5) Stops: A categorical feature with 3 distinct values that stores the number of stops between the source and destination cities.
6) Arrival Time: This is a derived categorical feature created by grouping time intervals into bins. It has six distinct time labels and keeps information about the arrival time.
7) Destination City: City where the flight will land. It is a categorical feature having 6 unique cities.
8) Class: A categorical feature that contains information on seat class; it has two distinct values: Business and Economy.
9) Duration: A continuous feature that displays the overall amount of time it takes to travel between cities in hours.
10)Days Left: This is a derived characteristic that is calculated by subtracting the trip date by the booking date.
11) Price: Target variable stores information of the ticket price.

In [279]:
# Ensure the necessary libraries are installed
try:
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    import sklearn
except ImportError:
    print("Installing required libraries...")
    %pip install -r requirements.txt
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    import sklearn
%matplotlib inline    


In [280]:
df = pd.read_excel('flight_price.xlsx')
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [281]:
# get the basic info of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [282]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


Feature Engineering

In [283]:
# Fix Date_of_Journey  Field
df['Date'] = df['Date_of_Journey'].str.split('/').str[0]
df['Month'] = df['Date_of_Journey'].str.split('/').str[1]
df['Year'] = df['Date_of_Journey'].str.split('/').str[2]

# Convert the new fields from type object to Int
df['Date'] = df['Date'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Year'] = df['Year'].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
 11  Date             10683 non-null  int64 
 12  Month            10683 non-null  int64 
 13  Year             10683 non-null  int64 
dtypes: int64(4), object(10)
memory usage: 1.1+ MB


In [284]:
# Drop Date Of Journey
try:
    df.drop('Date_of_Journey', axis=1, inplace=True)
except KeyError:
    print("Field removed previously")
    pass    
df.head

<bound method NDFrame.head of            Airline    Source Destination                  Route Dep_Time  \
0           IndiGo  Banglore   New Delhi              BLR → DEL    22:20   
1        Air India   Kolkata    Banglore  CCU → IXR → BBI → BLR    05:50   
2      Jet Airways     Delhi      Cochin  DEL → LKO → BOM → COK    09:25   
3           IndiGo   Kolkata    Banglore        CCU → NAG → BLR    18:05   
4           IndiGo  Banglore   New Delhi        BLR → NAG → DEL    16:50   
...            ...       ...         ...                    ...      ...   
10678     Air Asia   Kolkata    Banglore              CCU → BLR    19:55   
10679    Air India   Kolkata    Banglore              CCU → BLR    20:45   
10680  Jet Airways  Banglore       Delhi              BLR → DEL    08:20   
10681      Vistara  Banglore   New Delhi              BLR → DEL    11:30   
10682    Air India     Delhi      Cochin  DEL → GOI → BOM → COK    10:55   

       Arrival_Time Duration Total_Stops Additional_Info 

In [285]:
# Fix Arrival_Time  Field
df['Arrival_Time'] = df['Arrival_Time'].apply(lambda x:x.split(' ')[0])  # Remove all date entry and only keep hours and minutes
df.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25,19h,2 stops,No info,13882,9,6,2019
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,2019
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,2019


In [286]:
# Drop Departure and Arrival columns
try:
    df['Dep_hour'] = df['Dep_Time'].str.split(':').str[0]
    df['Dep_min'] = df['Dep_Time'].str.split(':').str[0]
    df['Arrival_hour'] = df['Arrival_Time'].str.split(':').str[0]
    df['Arrival_min'] = df['Arrival_Time'].str.split(':').str[0]

    df['Dep_hour'] = df['Dep_hour'].astype(int)
    df['Dep_min'] = df['Dep_min'].astype(int)
    df['Arrival_hour'] = df['Arrival_hour'].astype(int)
    df['Arrival_min'] = df['Arrival_min'].astype(int)
    
    df.drop(['Dep_Time','Arrival_Time'], axis =1, inplace=True)
except KeyError:
    print("Field processed previously")
    pass    
df.head


<bound method NDFrame.head of            Airline    Source Destination                  Route Duration  \
0           IndiGo  Banglore   New Delhi              BLR → DEL   2h 50m   
1        Air India   Kolkata    Banglore  CCU → IXR → BBI → BLR   7h 25m   
2      Jet Airways     Delhi      Cochin  DEL → LKO → BOM → COK      19h   
3           IndiGo   Kolkata    Banglore        CCU → NAG → BLR   5h 25m   
4           IndiGo  Banglore   New Delhi        BLR → NAG → DEL   4h 45m   
...            ...       ...         ...                    ...      ...   
10678     Air Asia   Kolkata    Banglore              CCU → BLR   2h 30m   
10679    Air India   Kolkata    Banglore              CCU → BLR   2h 35m   
10680  Jet Airways  Banglore       Delhi              BLR → DEL       3h   
10681      Vistara  Banglore   New Delhi              BLR → DEL   2h 40m   
10682    Air India     Delhi      Cochin  DEL → GOI → BOM → COK   8h 20m   

      Total_Stops Additional_Info  Price  Date  Month  Ye

In [287]:
# Split Duration : Complex as sometimes just minutes or hours are present

def split_time(time_str):
    hours = 0
    min = 0
    if 'h' in time_str:
        hours = int(time_str.split('h')[0])
        time_str = time_str.split('h')[1]
    if 'm' in time_str:
        min = int(time_str.split('m')[0])
    return hours, min
df[['Duration_hours', 'Duration_min']] = df['Duration'].apply(lambda x: pd.Series(split_time(x)))

In [288]:
# Fix Total_stops for missing value also map it as numarical
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [289]:
df['Total_Stops'].mode()  # We will use the result to replace the missing values with
df['Total_Stops'].unique()  # We see that there is a single null value

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [290]:
df['Total_Stops'] = df['Total_Stops'].map({'non-stop': 0, '2 stops': 2, '1 stop' : 1, '3 stops': 3,'4 stops':4, np.nan:1})
df.head(2)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,0,No info,3897,24,3,2019,22,22,1,1,2,50
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2,No info,7662,1,5,2019,5,5,13,13,7,25


In [291]:
print(df['Airline'].unique())
print(df['Source'].unique())
print(df['Destination'].unique())
print(df['Additional_Info'].unique())

['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']
['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']


<!-- While developing Data Science Projects, we may find datasets containing mixed data types consisting of both categorical as well as numerical columns. However, various Machine Learning models do not work with categorical data and to fit this data into the machine learning model it needs to be converted into numerical data. For example, suppose a dataset has a Gender column with categorical elements like Male and  Female. These labels have no specific order of preference and also since the data is string labels, machine learning models misinterpreted that there is some sort of hierarchy in them. -->

<!-- To address this issue, one effective technique is one hot encoding. OHE in machine learning transforms categorical data into a numerical format that machine learning algorithms can process without imposing any ordinal relationships. -->

One Hot Encoding for dimensional values

While developing Data Science Projects, we may find datasets containing mixed data types consisting of both categorical as well as numerical columns. However, various Machine Learning models do not work with categorical data and to fit this data into the machine learning model it needs to be converted into numerical data. For example, suppose a dataset has a Gender column with categorical elements like Male and  Female. These labels have no specific order of preference and also since the data is string labels, machine learning models misinterpreted that there is some sort of hierarchy in them.

To address this issue, one effective technique is one hot encoding. OHE in machine learning transforms categorical data into a numerical format that machine learning algorithms can process without imposing any ordinal relationships.

In [292]:
# Method to One hot encode features and drop orignal
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]]).astype(int)
    df = pd.concat([original_dataframe, dummies], axis=1)
    df = df.drop([feature_to_encode], axis=1)
    return(df)

In [293]:
# Loop features
features_to_encode = ['Airline','Source','Destination', 'Additional_Info']
for feature in features_to_encode:
    df = encode_and_bind(df, feature)
df.head()

,Route,Duration,Total_Stops,Price,Date,Month,Year,Dep_hour,Dep_min,Arrival_hour,...,Additional_Info_1 Long layover,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,BLR → DEL,2h 50m,0,3897,24,3,2019,22,22,1,...,0,0,0,0,0,0,0,0,1,0
1,CCU → IXR → BBI → BLR,7h 25m,2,7662,1,5,2019,5,5,13,...,0,0,0,0,0,0,0,0,1,0
2,DEL → LKO → BOM → COK,19h,2,13882,9,6,2019,9,9,4,...,0,0,0,0,0,0,0,0,1,0
3,CCU → NAG → BLR,5h 25m,1,6218,12,5,2019,18,18,23,...,0,0,0,0,0,0,0,0,1,0
4,BLR → NAG → DEL,4h 45m,1,13302,1,3,2019,16,16,21,...,0,0,0,0,0,0,0,0,1,0


In [294]:
df.drop(columns=['Route','Duration'], axis =1, inplace=True)  # Route is not required
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 44 columns):
 #   Column                                        Non-Null Count  Dtype
---  ------                                        --------------  -----
 0   Total_Stops                                   10683 non-null  int64
 1   Price                                         10683 non-null  int64
 2   Date                                          10683 non-null  int64
 3   Month                                         10683 non-null  int64
 4   Year                                          10683 non-null  int64
 5   Dep_hour                                      10683 non-null  int64
 6   Dep_min                                       10683 non-null  int64
 7   Arrival_hour                                  10683 non-null  int64
 8   Arrival_min                                   10683 non-null  int64
 9   Duration_hours                                10683 non-null  int64
 10  Duration_m

In [295]:
df.to_csv('flight_price_cleaned.csv', mode='w', index=False)
print("Data cleaned and saved to csv file")  # Print a success message

Data cleaned and saved to csv file


The dataset is now ready to be consumed for analysis/machine learning